In [ ]:
# YoloV5 [train] 예제
# https://www.kaggle.com/orkatz2/yolov5-train

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
'''
/ : 루트 (가장 최상의 디렉토리로 이동한다.)
./ : 현재 위치 (파일의 현재 디렉토리를 의미한다.)
../ : 현재 위치의 상단 폴더 (상위 디렉토리로 이동한다.)

ex) index.php가 C:\index\a에 위치한다면,
      여기서 / 는 C:
               ./ 는 a
              ../ 는 index라는 것.
    만약 두단계 상위 디렉토리로 이동하려면 '../../' 이렇게 쓰면 된다.
'''


df = pd.read_csv('./data/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
from tqdm.auto import tqdm
import shutil as sh
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

In [4]:
df.head()

,image_id,x,y,w,h,x_center,y_center,classes
0,b6ab77fd7,834.0,222.0,56.0,36.0,862.0,240.0,0
1,b6ab77fd7,226.0,548.0,130.0,58.0,291.0,577.0,0
2,b6ab77fd7,377.0,504.0,74.0,160.0,414.0,584.0,0
3,b6ab77fd7,834.0,95.0,109.0,107.0,888.5,148.5,0
4,b6ab77fd7,26.0,144.0,124.0,117.0,88.0,202.5,0


In [5]:
index = list(set(df.image_id))

In [8]:
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]    # 전체 학습데이터 중 5분의 1을 검증데이터로 나누는 부분인듯
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            
            # 폴더생성
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            
            # 폴더생성
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            
            # 생성한 폴더에 이미지를 복사한다.
            sh.copy("./data/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [5]:
'''

cfg?
cfg 파일은 신경망의 구조(layer 개수, 입력 데이터의 차원 등)를 명시한 파일이고 weights 파일은 실제로 학습된 신경망의 weight 값들을 저장한 것이다.
cfg = config = configuration, 구성

Apex?
NVIDIA APEX, 엔비디아에서 제공하는 더 빠른 학습방법에 대한 패키지
https://cvml.tistory.com/8

'''


!python ./yolov5/train.py --img 1024 --batch 1 --epochs 1 --data './yolov5/input/configYolo5/wheat0.yaml' --cfg './yolov5/input/configYolo5/yolov5x.yaml' --name yolov5x_fold0

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=1, bucket='', cache_images=False, cfg='./yolov5/input/configYolo5/yolov5x.yaml', data='./yolov5/input/configYolo5/wheat0.yaml', device='', epochs=1, evolve=False, img_size=[1024], multi_scale=False, name='yolov5x_fold0', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='')
Using CUDA device0 _CudaDeviceProperties(name='GeForce RTX 2070 SUPER', total_memory=7974MB)

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/

              from  n    params  module                                  arguments               

In [ ]:
# 오픈소스코드에는 !python 라인에 따옴표가 없었다.
# YoloV5에는 Cutmix, mosaic, brightness-contrast 등 최신 데이터 증강기법이 기적용되어 있어 또 할 필요는 없다.

In [ ]:
#Be careful that this command index = list(set(df.image_id)) is not sorted and we may get different order (e.g. I got different order when I run with Colab and Kaggle), so fold=0 or 1 may be does not matter.